In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [2]:
import os
import cv2
import yaml

# Base dataset path
base_path = '/mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset'
splits = ['train', 'val', 'test']

# Get class names from train/images subfolders
train_images_path = os.path.join(base_path, 'train', 'images')
class_names = sorted([d for d in os.listdir(train_images_path) if os.path.isdir(os.path.join(train_images_path, d))])
class_to_id = {name: idx for idx, name in enumerate(class_names)}

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Process each split
for split in splits:
    for class_name in class_names:
        image_dir = os.path.join(base_path, split, 'images', class_name)
        label_dir = os.path.join(base_path, split, 'labels', class_name)
        ensure_dir(label_dir)

        if not os.path.exists(image_dir):
            print(f"Skipping missing folder: {image_dir}")
            continue

        for img_file in os.listdir(image_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            img_path = os.path.join(image_dir, img_file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Warning: couldn't read image {img_path}")
                continue

            h, w, _ = image.shape

            # Dummy bounding box: center of image, 50% size
            x_center = 0.5
            y_center = 0.5
            bbox_width = 0.5
            bbox_height = 0.5
            class_id = class_to_id[class_name]

            # Write YOLO label file
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

print("✅ Labels generated for all splits.")

# Create dataset.yaml file
yaml_data = {
    'path': os.path.abspath(base_path),
    'train': os.path.abspath(os.path.join(base_path, 'train')),
    'val': os.path.abspath(os.path.join(base_path, 'val')),
    'test': os.path.abspath(os.path.join(base_path, 'test')),
    'nc': len(class_names),
    'names': class_names
}

with open('emotion_dataset.yaml', 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print("✅ 'emotion_dataset.yaml' created.")


Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/angry
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/boring
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/disgust
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/fear
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/happy
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/neutral
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/sad
Skipping missing folder: /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val/images/stress
Skipping missing folder: /mnt/d/Group50/Facia

In [6]:
import os
import shutil
import random

# Base dataset path
base_dir = '/mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset'

# Ratios
split_ratios = {
    'train': 0.7,
    'val': 0.15,
    'test': 0.15
}

# Source folders
image_root = os.path.join(base_dir, 'train', 'images')
label_root = os.path.join(base_dir, 'train', 'labels')

# Classes (subfolders)
class_names = [d for d in os.listdir(image_root) if os.path.isdir(os.path.join(image_root, d))]

for cls in class_names:
    img_class_dir = os.path.join(image_root, cls)
    lbl_class_dir = os.path.join(label_root, cls)

    images = [f for f in os.listdir(img_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    total = len(images)
    n_train = int(split_ratios['train'] * total)
    n_val = int(split_ratios['val'] * total)

    split_map = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:]
    }

    for split, files in split_map.items():
        img_dest_dir = os.path.join(base_dir, split, 'images', cls)
        lbl_dest_dir = os.path.join(base_dir, split, 'labels', cls)

        os.makedirs(img_dest_dir, exist_ok=True)
        os.makedirs(lbl_dest_dir, exist_ok=True)

        moved = 0
        for file in files:
            base_name, _ = os.path.splitext(file)
            label_file = base_name + '.txt'

            img_src = os.path.join(img_class_dir, file)
            lbl_src = os.path.join(lbl_class_dir, label_file)

            img_dst = os.path.join(img_dest_dir, file)
            lbl_dst = os.path.join(lbl_dest_dir, label_file)

            if os.path.exists(img_src):
                shutil.move(img_src, img_dst)
            else:
                print(f"❌ Missing image: {img_src}")
                continue

            if os.path.exists(lbl_src):
                shutil.move(lbl_src, lbl_dst)
            else:
                print(f"⚠️ Missing label: {lbl_src}")

            moved += 1

        print(f"✅ {split.upper()} - {cls}: Moved {moved} images.")

print("\n🎉 Dataset split complete with subfolders preserved.")


✅ TRAIN - angry: Moved 3502 images.
✅ VAL - angry: Moved 750 images.
✅ TEST - angry: Moved 751 images.
✅ TRAIN - boring: Moved 1295 images.
✅ VAL - boring: Moved 277 images.
✅ TEST - boring: Moved 279 images.
✅ TRAIN - disgust: Moved 2961 images.
✅ VAL - disgust: Moved 634 images.
✅ TEST - disgust: Moved 635 images.
✅ TRAIN - fear: Moved 3545 images.
✅ VAL - fear: Moved 759 images.
✅ TEST - fear: Moved 761 images.
✅ TRAIN - happy: Moved 3554 images.
✅ VAL - happy: Moved 761 images.
✅ TEST - happy: Moved 763 images.
✅ TRAIN - neutral: Moved 3528 images.
✅ VAL - neutral: Moved 756 images.
✅ TEST - neutral: Moved 756 images.
✅ TRAIN - sad: Moved 3570 images.
✅ VAL - sad: Moved 765 images.
✅ TEST - sad: Moved 765 images.
✅ TRAIN - stress: Moved 4249 images.
✅ VAL - stress: Moved 910 images.
✅ TEST - stress: Moved 912 images.
✅ TRAIN - suprised: Moved 3570 images.
✅ VAL - suprised: Moved 765 images.
✅ TEST - suprised: Moved 765 images.

🎉 Dataset split complete with subfolders preserved.


In [1]:
!pip install -U ultralytics

  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_6

In [1]:


from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-cls.pt")  # load a pretrained model (recommended for training)

# Train using the single most idle GPU
results = model.train(data="/mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset", epochs=100, imgsz=480, device=0,batch=-1)

Ultralytics 8.3.144 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

train: Scanning /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/train... 29774 images, 0 corrupt: 100%|██████████| 29774/29774 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 3.7±5.0 ms, read: 5.3±1.0 MB/s, size: 13.5 KB)


val: Scanning /mnt/d/Group50/Facial_Emotion_Recongnition/Datasets/Preprpcess_Large_Dataset/val... 6377 images, 0 corrupt: 100%|██████████| 6377/6377 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0013671875000000001), 40 bias(decay=0.0)
Image sizes 480 train, 480 val
Using 8 dataloader workers
Logging results to runs/classify/train4
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


  0%|          | 0/171 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 43, in do_one_step
    data = pin_memory(data, device)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 77, in pin_memory
    {k: pin_memory(sample, device) for k, sample in data.items()}
  File "/home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 77, in <dictcomp>
    {k: pin_memory(sample, device) for k, sample in data.items()}
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/group50/anaconda3/envs/condaenv/lib/python3.11/site-packages/torch/utils/data/_utils/pin_memory.py", line 66, in pin_memory
    return data.pin_memory(device)
           ^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: CUDA error:

In [ ]:
model.export(format="onnx")

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("E:\AppsTechnologies\AI\BeMA\Facial_Recog\\best.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["E:\AppsTechnologies\AI\BeMA\Facial_Recog\img.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [ ]:
import cv2
import time
from ultralytics import YOLO

# Define your emotion class names
emotion_classes = ['angry','boring','disgust','fear','happy','neutral','sad','stress','surprise']

# Load your custom model
model = YOLO("D:\RuhunaNew\Academic\Research\Facial_Recog\Group_50\\best.pt")

cap = cv2.VideoCapture(0)
inference_interval = 5  # seconds
last_inference_time = 0
last_label = "No emotion detected"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    # Run inference every 5 seconds
    if current_time - last_inference_time >= inference_interval:
        results = model(frame)
        last_inference_time = current_time

        # Process results
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            box = results[0].boxes[0]
            class_id = int(box.cls[0])
            confidence = box.conf[0].item()
            last_label = f"{emotion_classes[class_id]} ({confidence:.2f})"
            # Optionally, draw box:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, last_label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        else:
            last_label = "No emotion detected"

    # Overlay the last detected label on the frame
    cv2.putText(frame, last_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("YOLOv11 Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("D:\\RuhunaNew\\Academic\\Research\\Facial_Recog\\best.pt")

# Run inference on the source
results = model(source=0, stream=True)  # generator of Results objects

# Process the results
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    print(boxes)